# **Reading Annotations file**

In [1]:
#Importing drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#Displayig working directory
import os
os.getcwd()

'/content'

In [0]:
#Changing working directory
root_path = 'gdrive/My Drive/DL human pose '

os.chdir(root_path)

In [4]:
#Unzip the annotation file
import zipfile
filename = 'mpii_human_pose_v1_u12_2.zip'
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

#Confirm the creation of unzipped file
os.listdir()

#changing directory to annotation file path after unzipping
os.chdir('mpii_human_pose_v1_u12_2')

#List the files under annotation folder
os.listdir()

['bsd.txt', 'mpii_human_pose_v1_u12_1.mat', 'README.md']

In [5]:
#Loading .mat annotation file
from scipy.io import loadmat

matph = './mpii_human_pose_v1_u12_1.mat'
mat = loadmat(matph, struct_as_record=False)
release = mat['RELEASE'] 
print(release, type(release), release.shape)
object1 = release[0,0]
print(object1._fieldnames)

[[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f44bcab0cf8>]] <class 'numpy.ndarray'> (1, 1)
['annolist', 'img_train', 'version', 'single_person', 'act', 'video_list']


# **Labels Extraction**

In [0]:
#We will display the contents of all structs but at the end we will focus on act struct to get activity label and annolist to get image
#name
annolist = object1.__dict__['annolist']
img_train = object1.__dict__['img_train']
ver = object1.__dict__['version']
single_person = object1.__dict__['single_person']
activity = object1.__dict__['act']
video_list = object1.__dict__['video_list'] 

In [7]:
#Exploring activity structure noting that at the end we need to extract act_name
print(activity, type(activity), activity.shape)
print(activity[5,0]._fieldnames)
activity[5,0].__dict__['cat_name'][0]

[[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f4491386860>]
 ...
 [<scipy.io.matlab.mio5_params.mat_struct object at 0x7f448f2e2a58>]] <class 'numpy.ndarray'> (24987, 1)
['cat_name', 'act_name', 'act_id']


'sports'

In [8]:
#Exploring annolist structure noting that at the end we need to extract image name
print(annolist, type(annolist), annolist.shape)
print(annolist[0,5]._fieldnames)
img=annolist[0,5].__dict__['image']
img[0,0].__dict__['name'][0]

[[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f44bcabaef0>
  <scipy.io.matlab.mio5_params.mat_struct object at 0x7f44bc829198> ...
  <scipy.io.matlab.mio5_params.mat_struct object at 0x7f4491386668>]] <class 'numpy.ndarray'> (1, 24987)
['image', 'annorect', 'frame_sec', 'vididx']


'015599452.jpg'

In [0]:
#Now let's get list with the labels of all images and other list for iamges names noting that for images assigned for testing the activity
#was left empty so we will assign the word 'test' for the images that has no activity
import numpy as np
label=[]
image=[]
for i in range(len(activity)):
  img=annolist[0,i].__dict__['image']
  image.append(img[0,0].__dict__['name'][0])
  if len(activity[i,0].__dict__['cat_name'])>0:
    label.append(activity[i,0].__dict__['cat_name'][0])
  else:
    label.append("test")

# **Labels Exploration**

In [11]:
#Importing libraries
import numpy as np
import pandas as pd
#It is better to deal with pandas dataframe so we will convert image,label to 2 columns in pandas dataframe
df_label = pd.DataFrame(list(zip(image, label)), 
               columns =['image', 'label']) 

#Let's output the first 5 rows in the dataframe
df_label.head(10)

,image,label
0,037454012.jpg,test
1,095071431.jpg,test
2,073199394.jpg,test
3,059865848.jpg,test
4,015601864.jpg,sports
5,015599452.jpg,sports
6,005808361.jpg,sports
7,086617615.jpg,sports
8,060111501.jpg,sports
9,070807258.jpg,sports


In [0]:
#sorting the label dataframe based on the image name ascending
df_label = df_label.sort_values(by = 'image')
df_label = df_label.reset_index(drop=True)
df_label.head(10)

,image,label
0,000001163.jpg,volunteer activities
1,000003072.jpg,sports
2,000004812.jpg,conditioning exercise
3,000005283.jpg,walking
4,000013469.jpg,home activities
5,000015774.jpg,sports
6,000022704.jpg,sports
7,000024087.jpg,test
8,000024100.jpg,miscellaneous
9,000024293.jpg,test


In [16]:
#Now let's display the number of images that are used as test which we will remove later as it is useless
#to include unlabelled data unless that we will use it as unsupervised data for pretraining.
pd.value_counts(df_label[df_label.label == 'test']['label'])

test    6954
Name: label, dtype: int64

In [14]:
#Displaying number of images for each catgory
pd.value_counts(df_label[df_label.label != 'test']['label'])

sports                    3593
occupation                2412
conditioning exercise     1755
home activities           1416
lawn and garden           1130
home repair               1090
water activities          1066
winter activities          778
miscellaneous              755
fishing and hunting        725
dancing                    722
walking                    608
music playing              583
bicycling                  533
running                    316
inactivity quiet/light     189
religious activities       124
transportation              93
self care                   90
volunteer activities        55
Name: label, dtype: int64

We can notice that 6954 images were reserved as a test dataset which we will need to remove as we don't know the exact labels for these images and separate other images to training and validation to evaluate our model.
Also it was observed that the different activities are not equally distributed in the dataset so we need to make stratified sampling during train_test_split to assure that classes distribution is the same in training and validation data

# **Saving lables to csv**

In [0]:
#saving a local copy of label dataframe as csv format to load it directly every time we run the code, to avoid wasting time doing all preprocessing from scratch
os.chdir(root_path1)
df_label.to_csv('names_label.csv', index=False)